In [1]:
import pandas as pd

N = 100_000_000
separator = '\t'
base_folder = 'data/'
group_folder = 'MichelLardon/'
file = "2"

dsource_file = base_folder + 'donnees_sources'
dtarget_file = base_folder + group_folder + 'donnees_anonymes' + file

attack_file = base_folder + group_folder + 'fichier_attaque' + file + '.json'

In [2]:
dsource = pd.read_pickle(dsource_file + '.pkl')

In [3]:
dtarget = pd.read_pickle(dtarget_file + '.pkl')

In [ ]:

from sklearn.ensemble import RandomForestClassifier

df = pd.DataFrame(columns=['source_id', 'week_year', 'target_id'])

# Iterate over unique week_year values
for index, week_year in enumerate(dsource['week_year'].unique()):
    print(index, ':', week_year)

    X = ['timestamp', 'distance']
    # X = ['timestamp', 'lat', 'long']
    y = ['id']

    # Filter data for the current week_year
    src = dsource[dsource['week_year'] == week_year]
    tgt = dtarget[dtarget['week_year'] == week_year]

    clf = RandomForestClassifier(n_estimators=10, random_state=42, n_jobs=6)

    # Fit the model using partial_fit
    print('Fitting...')
    clf.fit(tgt[X], tgt[y])

    print('Predicting...')
    index_list = clf.predict(src[X])

    source_ids = src[y].squeeze().reset_index(drop=True)
    target_ids = index_list.squeeze()

    # dfi = pd.DataFrame({'source_id': source_ids.tolist(), 'week_year': week_year, 'target_id': tgt[y].squeeze()})
    print('Creating DataFrame...')
    dfi = pd.DataFrame({'source_id': source_ids, 'week_year': week_year, 'target_id': target_ids})

    # Append the current week's DataFrame to the main DataFrame
    df = pd.concat([df, dfi], ignore_index=True)

In [22]:
df

,source_id,week_year,target_id
0,1,2015-09,1494857
1,1,2015-09,1494857
2,1,2015-09,1494857
3,1,2015-09,1494857
4,1,2015-09,1494857
...,...,...,...
3430294,110,2015-09,4573307
3430295,110,2015-09,4573307
3430296,110,2015-09,4573307
3430297,110,2015-09,4573307


In [44]:
occurrences = df.groupby(['target_id', 'week_year', 'source_id']).size().reset_index(name='occurrences')
df = df.merge(occurrences, on=['source_id', 'week_year', 'target_id'], how='left')
df.head(10)

,source_id,week_year,target_id,occurrences
0,1,2015-09,1494857,1968
1,1,2015-09,1494857,1968
2,1,2015-09,1494857,1968
3,1,2015-09,1494857,1968
4,1,2015-09,1494857,1968
5,1,2015-09,1494857,1968
6,1,2015-09,1494857,1968
7,1,2015-09,1494857,1968
8,1,2015-09,1494857,1968
9,1,2015-09,1494857,1968


Calcul du nombre total d'apparition de chaque source_id

In [45]:
total_count = df.groupby('source_id').size().reset_index(name='total_count')
df = df.merge(total_count, on='source_id', how='left')
df.head(10)

,source_id,week_year,target_id,occurrences,total_count
0,1,2015-09,1494857,1968,863885
1,1,2015-09,1494857,1968,863885
2,1,2015-09,1494857,1968,863885
3,1,2015-09,1494857,1968,863885
4,1,2015-09,1494857,1968,863885
5,1,2015-09,1494857,1968,863885
6,1,2015-09,1494857,1968,863885
7,1,2015-09,1494857,1968,863885
8,1,2015-09,1494857,1968,863885
9,1,2015-09,1494857,1968,863885


Calcul du pourcentage d'apparition d'un id source pour chaque semaine

In [46]:
df["percentage"] = df["occurrences"] / df["total_count"]
df = df.drop(['occurrences', 'total_count'], axis=1)
df.head(10)

,source_id,week_year,target_id,percentage
0,1,2015-09,1494857,0.002278
1,1,2015-09,1494857,0.002278
2,1,2015-09,1494857,0.002278
3,1,2015-09,1494857,0.002278
4,1,2015-09,1494857,0.002278
5,1,2015-09,1494857,0.002278
6,1,2015-09,1494857,0.002278
7,1,2015-09,1494857,0.002278
8,1,2015-09,1494857,0.002278
9,1,2015-09,1494857,0.002278


Suppression des duplicatas

In [47]:
df = df.drop_duplicates()
df.head(10)

,source_id,week_year,target_id,percentage
0,1,2015-09,1494857,0.002278
53,1,2015-09,225503,0.001770
76,1,2015-09,1829209,0.006296
241,1,2015-09,7402473,0.001052
313,1,2015-09,7319518,0.000164
328,1,2015-09,1125336,0.000208
413,1,2015-09,6609548,0.001593
593,1,2015-09,1461127,0.001072
613,1,2015-09,4386281,0.003590
629,1,2015-09,8526733,0.000832


In [48]:
import joblib

try:
    encoder = joblib.load(base_folder + group_folder + 'LabelEncoder' + file + '.joblib')
    df['target_id'] = encoder.inverse_transform(df['target_id'].tolist())
except:
    pass

In [49]:
result_dict = {}
e = df

while e.shape[0] != 0:
    e = e.sort_values(['percentage'], ascending=False)

    row = e.iloc[0]
    ano_id = row['target_id']
    week_year = row['week_year']
    src_id = row['source_id']

    if str(src_id) not in result_dict:
        result_dict[str(src_id)] = {week_year: [str(ano_id)]}
    else:
        result_dict[str(src_id)][week_year] = [str(ano_id)]

    e = e[~((e['target_id'] == ano_id) & (e['week_year'] == week_year))]
    e = e[~((e['source_id'] == src_id) & (e['week_year'] == week_year))]

result_dict

{'58': {'2015-09': ['6290814'], '2015-10': ['3437488']},
 '110': {'2015-09': ['2295226'], '2015-10': ['8290989']},
 '71': {'2015-15': ['3347588'],
  '2015-09': ['1494857'],
  '2015-17': ['1199841'],
  '2015-16': ['5305907'],
  '2015-11': ['5275813'],
  '2015-13': ['6271743'],
  '2015-12': ['6344403']},
 '83': {'2015-13': ['8956735'],
  '2015-10': ['7408315'],
  '2015-17': ['735602'],
  '2015-12': ['995718'],
  '2015-09': ['9607278'],
  '2015-19': ['5610465'],
  '2015-18': ['5610465'],
  '2015-14': ['8956735'],
  '2015-15': ['2605209']},
 '39': {'2015-15': ['9564600'],
  '2015-18': ['8165380'],
  '2015-09': ['8525546'],
  '2015-14': ['2757835']},
 '87': {'2015-11': ['710077'],
  '2015-10': ['3519774'],
  '2015-09': ['5020041'],
  '2015-17': ['4834555'],
  '2015-18': ['4834555'],
  '2015-16': ['2612459']},
 '14': {'2015-09': ['4643626'],
  '2015-10': ['3378265'],
  '2015-11': ['3671339']},
 '55': {'2015-09': ['3659973'], '2015-10': ['573306'], '2015-11': ['573306']},
 '23': {'2015-09': [

In [50]:
from collections import OrderedDict

ordered_dict = OrderedDict(sorted((int(key), value) for key, value in result_dict.items()))

for key, value in ordered_dict.items():
    ordered_dict[key] = OrderedDict(sorted(value.items()))

In [51]:
import json

result_json = json.dumps(ordered_dict, indent=2)
with open(attack_file, 'w') as file:
    file.write(result_json)